In [5]:
import pandas as pd
import numpy as np

import yfinance as yf

import seaborn as sns
import matplotlib.pyplot as plt

KeyboardInterrupt: 

In [ ]:
# Banking Stocks: JPMorgan and Chase, GoldmanSachs, Citigroup, Bank of America
symbols = ["JPM", "GS", "C", "BAC"]